# Hyperparameter optimization

This notebook implements the entire processing pipeline to find optimal hyperparameters. It is composed of the following components:
1. Text preprocessor that filters and normalizes the input ([`Filter` + `TextPreprocessor`](#Text-Preprocessing))
2. An adapter that allows easy iteration over the content of Pandas DataFrames ([`Corpus_adapter`](#Corpus-Adapter))
3. The [`Doc2Vec Transformer`](#Doc2Vec-Transformer) builds a document embedding model and transforms text pieces into their Doc2Vec representation.
4. The [`Doc2Vec-Classifier`](#Doc2Vec-Classifier) predicts the class/category of a document.
5. In the [`Pipeline-and-Hyperparameter-optimization`](#Pipeline-and-Hyperparameter-optimization) section the aforementioned pieces are put together and the processing is carried out.

You may have noticed that there is a certain interface design style (e.g. `fit()`, `transform()` and `predict()` methods). This is due to the framework provided by [scikit-learn](http://scikit-learn.org/stable/) which allows a simple automation of the hyperparameter search.

# Text Preprocessing

In [1]:
import os
import codecs
import unicodedata
import re
from nltk.stem.wordnet import WordNetLemmatizer

class Filter():
    def __init__(self, text=''):
        self.s = text
        self._lem = WordNetLemmatizer()

    def removeHeader(self):
        if '\n\n' in self.s:
            self.s = self.s[self.s.index('\n\n'):]
        return self
            
    def normalizeCaracters(self):
        # converts 'ueber' to 'uber'
        self.s = unicodedata.normalize("NFKD", self.s).encode("ascii", "ignore").decode("utf8")
        return self
    
    def selectCharacters(self):
        # Lets pass only meaningful characters 
        self.s = re.sub(r'([^a-zA-Z0-9 \-\_%])', '', self.s)
        return self
    
    def multipleDots(self):
        # removes multiple dots with optional whitespaces in between
        self.s = re.sub(r'((\.\s*){2,})', '', self.s)
        return self
    
    def multipleSpaces(self):
        # Removes multiple whitespaces
        self.s = re.sub(r'(\s{2,})', ' ', self.s)
        return self
    
    def lower(self):
        # lower cases everything
        self.s = self.s.lower()
        return self
    
    def shortTokens(self):
        # removes tokens shorter than minLen
        self.s = re.sub(r'(?<=\s)[\w?!%,.;:\/]{1,3}(?=\s|\Z)', '', self.s)
        return self
        
    def digits(self):
        # removes all digits except digits that represent years
        self.s = re.sub(r'\b(?!(\D\S*|[12][0-9]{3})\b)\S+\b', '', self.s)
        return self
    
    def removeHtml(self):
        self.s = re.sub(r'<.*?>', '', self.s)
        return self
    
    def removeMailAddresses(self):
        self.s = re.sub(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', '', self.s)
        return self
    
    def removeQuotes(self):
        self.s = re.sub(r'["\']', '', self.s)
        return self    
    
    def replaceNewline(self):
        self.s = self.s.replace('\n', ' ')
        return self
    
    def lemmatize(self):
        n = [self._lem.lemmatize(w, 'n') for w in self.s.split()]
        v = [self._lem.lemmatize(w, 'v') for w in self.s.split()]
        
        res = []
        for i, w in enumerate(self.s.split()):
            if w != n[i]:
                res.append(n[i])
            elif w != v[i]:
                res.append(v[i])
            else:
                res.append(w)
                
        self.s = ' '.join(res)
        return self
    
    def result(self):
        return self.s
    
    def __call__(self):
        # Calls the default usage of this class.
        return self.removeHeader()\
                    .removeHtml() \
                    .removeMailAddresses() \
                    .normalizeCaracters() \
                    .replaceNewline() \
                    .selectCharacters() \
                    .removeQuotes()\
                    .digits() \
                    .multipleDots() \
                    .lower() \
                    .shortTokens() \
                    .multipleSpaces() \
                    .result()


class TextPreprocessor():    
    """
    A tiny wrapper around the Filter class, to comply with the sklearn interface
    """
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        res = []
        for x in X:
            res.append(Filter(x)())
        return res
    
    def get_params(self, deep=True):
        return {}
    
    def set_params(self, **params):
        return self

# Corpus Adapter

The `Corpus_adapter` class wraps around the pandas dataframe. It creates a sub dataset by randomly picking `sample_size` documents. Since this class implements the `__iter__` method, instances allow simple iteration over the sub dataset.

In [2]:
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split
import pandas as pd

class Corpus_adapter():
    def __init__(self, corpus, sample_size=10000, random_state=42):
        self.df = pd.read_pickle(corpus)
        
        assert sample_size >= 0, 'Sample size must be positive'
        if sample_size >= len(self.df):
            print('sample_size to large. will be set to max val: {}'.format(len(self.df)))
            sample_size = len(self.df)
        
        rnd = np.random.RandomState(random_state)
        self.sample = rnd.choice(self.df.index, sample_size, replace=False)
    
    def __iter__(self):
        for idx in self.sample:
            d = self.df.ix[idx]
            yield {
                    'idx': idx, 
                    'text': d['text'], 
                    'category': d['category']
                }

# Doc2Vec Transformer

This transformer class implements the methods necessary to be processed by `sklearn.pipeline.Pipeline` (`fit` and `transform`) as well as `sklearn.model_selection.GridSearchCV` (`get_params` and `set_params`).
It contains the Doc2Vec model which is trained by the `fit` method. The `transform`method accepts an array-like object containing text documents that are transformed to their vector representation.

An exhaustive description of the hyperparameters can be found [here](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.Doc2Vec).

__Note__: `set_params` and `get_params` have no effect on the model whatsoever. Their purpose is merely to conform to the interface `sklearn.model_selection.GridSearchCV` defines. Changing a model parameter requires re-training. Fortunately `GridSearchCV` instanciates for each configuration a new `D2VTransformer` and calls the `fit` method, which ensures appropriate behavior.

In [3]:
import os
import gensim
import logging
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
from collections import defaultdict, OrderedDict

class D2VTransformer():
    def __init__(self, **kwargs):
        self.params = OrderedDict({
                'size': 24,
                'window': 10,
                'workers': 1, 
                'min_count': 3, 
                'dm': 0,
                'hs': 0,
                'iter': 5,
                'negative': 10,
                'alpha': .025, 
                'min_alpha': 1e-4, 
                'batch_words': 1000,
                'seed': 42,
            })
        self.params.update(kwargs)
        self.doc_cache = {}
        self.model = None
        
    def fit(self, X, y):
        class IterList():
            def __init__(self, X):
                self.X = X
            def __iter__(self):
                return (TaggedDocument(words=d.split(), tags=[str(i)]) for i, d in enumerate(self.X))
        listIter = IterList(X)

        # We need to memorize the documents we've already seen (see transform method)
        for i, x in enumerate(X):
            self.doc_cache[hash(x)] = i
        self.model = Doc2Vec(**self.params)
        self.model.build_vocab(listIter)

        self.model.train(listIter, report_delay=60.0)

        self.model.docvecs.init_sims() # this initializes the syn0norm array
        return self

    def transform(self, X, alpha=1e-1):
        res = []
        for x in X:
            if hash(x) in self.doc_cache: # if seen during training
                res.append(self.model.docvecs.doctag_syn0norm[self.doc_cache[hash(x)]])
            else:
                res.append(self.model.infer_vector(x.split(), alpha=alpha))
        return np.array(res)
    
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        self.params.update(**params)
        return self

# Doc2Vec Classifier

This class implements a k-nearest neighbors classifier with cosine similarity distance metric (`predict` method). Furthermore, it implements the `score` method which determines the quality of the prediction with respect to a given dataset.

In [4]:
class D2VClassifier():
    def __init__(self, **kwargs):
        self.params = {
            'k': 10
        }
        self.params.update(kwargs)
    
    def fit(self, X, y):
        self.data = X
        self.targets = y
    
    def predict(self, X):
        dists = np.dot(X, self.data.transpose())
        best = np.fliplr(np.argsort(dists, axis=1))
        res = []
        for i, bestk in enumerate(best[:, 0:self.params['k']]):
            counter = defaultdict(int)
            for j, idx in enumerate(bestk):
                counter[self.targets[idx]] += dists[i][idx]
            counter = [(cat, val) for cat, val in counter.items()]
            res.append(sorted(counter, key=lambda x: -x[1])[0][0])
        return np.array(res)
    
    def score(self, X, y):
        pred = self.predict(X)
        return np.mean(pred == y)

    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        self.params.update(**params)
        return self

# Pipeline and Hyperparameter optimization

In the following section the previously discussed parts are coming together:
1. The dataset gets loaded.
2. A transformer-classifier pipeline is built.
3. The hyperparameter space gets defined.
4. The grid search is carried out

Since this part is crucial, a couple of things should be highlighted. The `Pipeline` class integrates multiple transformation steps with a concluding classification step into one object. The resulting object behaves just like a normal scikit-learn classifier with the exception that it accepts an unprocessed corpus. All the preprocessing is hidden within the `Pipeline` object.

The hyperpatemeter search (`GridSearchCV`) works on that classifier as well as a set of pre-defined hyperparameters which it is supposed to test for. `GridSearchCV` executes an exhaustive search on the entire parameter space which obviously limits the parameter space that can be searched or rather imposes demands that some machine won't comply with. For situations like these [`RandomizedSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) is more appropriate.

As the `CV` in `GridSearchCV` indicates, Cross-validation is also performed along the way.

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from itertools import product
import multiprocessing as mp
import pandas as pd
import numpy as np

def run_parametersearch(gs_params, corpus_path, jobs=4, sample_sizes=(int(1e4), int(1e3))):
    txt_clf = Pipeline([
            ('prepro', TextPreprocessor()),
            ('trans', D2VTransformer(workers=mp.cpu_count())),
            ('cls', D2VClassifier())
        ])

    res = {}
    for sample_size in sample_sizes:
        corpus_adapter = Corpus_adapter(corpus=corpus_path, sample_size=sample_size)
        gs_clf = GridSearchCV(txt_clf, gs_params, n_jobs=jobs, cv=5, verbose=1, return_train_score=False)

        X_train = [f['text'] for f in corpus_adapter]
        y_train = [f['category'] for f in corpus_adapter]
        res[sample_size] = gs_clf.fit(X_train, y_train)
    return res

In [6]:
def gs_to_df(gs_res):
    '''
    gs_to_df gathers the results of a gridsearch run into a pandas dataframe.
    gs_res is a dict who's keys are the sample sizes and who's values are GridSearchCV instances.
    '''
    df = None
    for sample_size, gs in gs_res.items():
        cv_report = gs.cv_results_
        
        mean = cv_report['mean_test_score']
        std = cv_report['std_test_score']
        params = cv_report['params']
        
        mean = pd.Series(mean, name='mean')
        std = pd.Series(std, name='std')
        sample_size = pd.DataFrame([sample_size]*len(params), columns=['sample_size'], dtype=int)
        params = pd.DataFrame((pd.Series(p) for p in params))
        
        params = params.merge(pd.DataFrame(sample_size), left_index=True, right_index=True)
        params = params.merge(pd.DataFrame(mean), left_index=True, right_index=True)
        params = params.merge(pd.DataFrame(std), left_index=True, right_index=True)
        
        if df is None:
            df = params
        else:
            df = df.append(params)
    df = df.reset_index()
    return df

# Corpus Configuration
The following configuration determines the input paths as well as the result paths for potentially multiple datasets. The input format was already defines in the introduction of one of the previous [notebooks](1.0_Amazon_corpus_to_pandas.ipynb#Amazon-review-corpus).
In case you don't define any of the `gs_params` they will be set to one sensible default value.

In [ ]:
jobs=4

experiment_data = {
    'sample_sizes': (1000, 10000),
    'corpora': {
        'amazon' : {
            'input_path': 'data/AMAZON/dataframes/amazon.pkl',
        },
        '20newsgroups' : {
            'input_path': 'data/20NEWSGROUPS/dataframes/20NEWSGROUPS.pkl',
        }   
    },
    'gs_params': { # This dict defines the parameters to test for
        'trans__size' : (24,),
        'trans__iter': (100,),
        'trans__alpha': (.01,),
        'trans__negative': (20,),
        'trans__window': (5,),
        'trans__hs': (1,),
        'trans__dm': (0,),
        'cls__k': (10,),
    },
    'param_labels': { # so python names are independent of names in the plots
        'trans__size' : r'd',
        'trans__iter': r'epoch',
        'trans__alpha': r'\alpha',
        'trans__negative': r'ns',
        'trans__window': r'win',
        'trans__hs': r'hs',
        'trans__dm': r'arch',
        'cls__k': r'k',        
    }
}
del experiment_data['corpora']['20newsgroups']

# Start grid search

In [ ]:
from datetime import datetime
import pickle

for corpus, config in experiment_data['corpora'].items():
    print('Processing {} corpus'.format(corpus))
    results = run_parametersearch(
        experiment_data['gs_params'], 
        config['input_path'], 
        jobs=jobs, 
        sample_sizes=experiment_data['sample_sizes']
    )
    df = gs_to_df(results)
    experiment_data['corpora'][corpus]['gs_results'] = df 

# serialize the results of this run
now = datetime.now()
with open('data/{}_gs_results.pkl'.format(now.strftime("%Y%m%d-%H:%M")), 'wb') as fh:
    pickle.dump(experiment_data, fh)

# Bayesian Optimization
The code that is presented in this section is based on the work of [Thomas Huijskens](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/). It implements Bayesian Optimization to find optimal hyperparameter values for the task at hand[1].

[1] Snoek, J., Larochelle, H., & Adams, R. P. (2012). Practical bayesian optimization of machine learning algorithms. In Advances in neural information processing systems (pp. 2951-2959).

In [ ]:
import numpy as np
import sklearn.gaussian_process as gp

from scipy.stats import norm
from scipy.optimize import minimize

def expected_improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):
    """ expected_improvement

    Expected improvement acquisition function.

    Arguments:
    ----------
        x: array-like, shape = [n_samples, n_hyperparams]
            The point for which the expected improvement needs to be computed.
        gaussian_process: GaussianProcessRegressor object.
            Gaussian process trained on previously evaluated hyperparameters.
        evaluated_loss: Numpy array.
            Numpy array that contains the values of the loss function for the previously
            evaluated hyperparameters.
        greater_is_better: Boolean.
            Boolean flag that indicates whether the loss function is to be maximised or minimised.
        n_params: int.
            Dimension of the hyperparameter space.

    """
    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0

    return expected_improvement

In [ ]:
import sklearn.gaussian_process as gp
from collections import OrderedDict

def gen_params(param_template, size=None):
    new_params = OrderedDict({})
    for param_name, gen_func in param_template.items():
        new_params[param_name] = gen_func(size)
    return new_params

def param_dict_to_numpy_array(params):
    return np.array([[v for k, v in p.items()] for p in params])

def bayesian_optimisation(n_iters, sample_loss, param_template, n_pre_samples=5,
                          gp_params=None, random_search=10, alpha=1e-5, epsilon=1e-7, verbose=False):
    """ bayesian_optimisation

    Uses Gaussian Processes to optimise the loss function `sample_loss`.

    Arguments:
    ----------
        n_iters: integer.
            Number of iterations to run the search algorithm.
        sample_loss: function.
            Function to be optimised.
        param_template: dict.
            Contains 'param_name': generator_function pairs
        x0: array-like, shape = [n_pre_samples, n_params].
            Array of initial points to sample the loss function for. If None, randomly
            samples from the loss function.
        n_pre_samples: integer.
            If x0 is None, samples `n_pre_samples` initial points from the loss function.
        gp_params: dictionary.
            Dictionary of parameters to pass on to the underlying Gaussian Process.
        random_search: integer.
            Flag that indicates whether to perform random search or L-BFGS-B optimisation
            over the acquisition function.
        alpha: double.
            Variance of the error term of the GP.
        epsilon: double.
            Precision tolerance for floats.
    """

    x_list = []
    y_list = []
    
    for i in range(n_pre_samples):
        new_params = gen_params(param_template)

        x_list.append(new_params)
        y_list.append(sample_loss(new_params))

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)
    if verbose:
        print('beginning optimization')
        
    for n in range(n_iters):
        xp = np.array(param_dict_to_numpy_array(x_list))
        yp = np.array(y_list)
        model.fit(xp, yp)

        # Sample next hyperparameter
        new_params = []
        for i in range(random_search):
            new_params.append(gen_params(param_template))
        x_random = param_dict_to_numpy_array(new_params)
        
        # x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
        ei = expected_improvement(x_random, model, yp, greater_is_better=True, n_params=x_random.shape[1])
        next_sample = new_params[np.argmax(ei)]

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(param_dict_to_numpy_array([next_sample]) - xp) <= epsilon):
            next_sample = gen_params(param_template)

        # Sample loss for new set of parameters
        cv_score = sample_loss(next_sample)
        
        if verbose:
            print('{} -> {}'.format(next_sample, cv_score))

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

In [ ]:
from sklearn.model_selection import cross_val_score

def sample_loss():
    def inner_loss(params, data=None, target=None):
        params = params.copy()
        clf_param = {'k': 10}
        if 'k' in params:
            clf_param['k'] = params.pop('k')
            
        txt_clf = Pipeline([
                ('prepro', TextPreprocessor()),
                ('trans', D2VTransformer(workers=mp.cpu_count(), **params)),
                ('cls', D2VClassifier(**clf_param))
            ])
        return cross_val_score(txt_clf, X=data, y=target, cv=5).mean()
    
    corpus_path = '../data/20NEWSGROUPS/dataframes/20NEWSGROUPS.pkl'
    corpus_adapter = Corpus_adapter(corpus=corpus_path, sample_size=18845)
    data = [f['text']  for f in corpus_adapter]
    target = [f['category'] for f in corpus_adapter]
    
    return partial(inner_loss, data=data, target=target)

In [ ]:
from functools import partial

param_template = OrderedDict({
    'size': (lambda size: np.random.randint(2, 251, size=size)),
    'window': (lambda size: np.random.randint(1, 51, size=size)),
    'negative': (lambda size: np.random.randint(1, 21, size=size)),        
    'alpha': (lambda size: 10 ** -np.random.uniform(0, 3, size=size)), 
    'iter': (lambda size: np.random.randint(3, 51, size=size)),        
    'dm': (lambda size: np.random.randint(0, 2, size=size)),
    'hs': (lambda size: np.random.randint(0, 2, size=size)),
    'k': (lambda size: np.random.randint(1, 50, size=size))
})

xp, yp = bayesian_optimisation(n_iters=100,
                               sample_loss=sample_loss(), 
                               param_template=param_template,
                               n_pre_samples=10,
                               random_search=10000,
                               verbose=True
                              )

### Inspecting the results

In [ ]:
for ix in list(reversed(yp.argsort()))[:10]:
    print("Accuracy: {} \n {}\n".format(yp[ix], xp[ix]))